<h5>Importing libraries</h5>

In [96]:
#data handling
import pandas as pd
import numpy as np
import os 
from PIL import Image

# Machine learning (non deep learning)
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 

# Deep learning
import tensorflow as tf
import keras
from keras import layers
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# misc
import time # for recording model training time

<h5>Read in images into numpy arrays and record type of tumor with one hot encoder</h5>

In [97]:
img_reshape_size = (32,32)

data = []
result = []

encoder = OneHotEncoder()
encoder.fit([[0], [1], [2], [3]])  # 0 = no tumor, 1 = meningioma tumor, 2 = glioma tumor , 3 = pitulitary tumor

# path to non tumor files
no_tumor_image_dir = "dataset/split_data/multi-class/no_tumor"
no_tumor_files = os.listdir(no_tumor_image_dir)

# path to meningioma files
meningioma_image_dir = "dataset/split_data/multi-class/meningioma_tumor"
meningioma_files = os.listdir(meningioma_image_dir)

# path to glioma files
glioma_image_dir = "dataset/split_data/multi-class/glioma_tumor"
glioma_files = os.listdir(glioma_image_dir)

# path to pitulitary files
pitulitary_image_dir = "dataset/split_data/multi-class/pitulitary_tumor"
pitulitary_files = os.listdir(pitulitary_image_dir)


print("lengths of the tumors files")
print("length of meningioma tumor files " + str(len(meningioma_files)))
print("length of glioma tumor files " + str(len(glioma_files)))
print("length of pitulitary tumor files " + str(len(pitulitary_files)))
print("length of non-tumor files " + str(len(no_tumor_files)))
print("total files is " + str(len(meningioma_files)+len(glioma_files)+len(pitulitary_files)+len(no_tumor_files)))
# should be 3000 files in total


# could be looped to reduce amount of code

## No tumor ##
for file in no_tumor_files:
    temp_file_path = no_tumor_image_dir + "/" +file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    data.append(np.array(img))
    result.append(encoder.transform([[0]]).toarray())

## Meningioma tumor ## 
for file in meningioma_files:
    temp_file_path = meningioma_image_dir + "/" + file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    data.append(np.array(img))
    result.append(encoder.transform([[1]]).toarray())
       
## Glioma tumor ##
for file in glioma_files:
    temp_file_path = glioma_image_dir + "/"  +file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    data.append(np.array(img))
    result.append(encoder.transform([[2]]).toarray())
        
## Pitulitary tumor ##     
for file in pitulitary_files:
    temp_file_path = pitulitary_image_dir + "/"  +file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    data.append(np.array(img))
    result.append(encoder.transform([[3]]).toarray())

lengths of the tumors files
length of meningioma tumor files 855
length of glioma tumor files 860
length of pitulitary tumor files 831
length of non-tumor files 454
total files is 3000


<h5>Reshape data and split between test and train data</h5>

In [98]:
data = np.array(data)
result = np.array(result)
result = result.reshape(3000,4)
x_train,x_test,y_train,y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

<h5>Build the CNN model</h5>

In [99]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(32, 32, 1), padding = 'Same'))
model.add(Conv2D(32, kernel_size=(2, 2),  activation ='relu', padding = 'Same'))


model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))
model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(loss = "categorical_crossentropy", optimizer='Adamax',  metrics = ['accuracy'])
#print(model.summary())

<h5>Train the model and record time to train</h5>

In [100]:
start_time = time.time()
history = model.fit(x_train, y_train, epochs = 300, batch_size = 2, verbose=1, validation_data = (x_test, y_test), callbacks=[callback])
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/300
1200/1200 [==============================] - 13s 10ms/step - loss: 1.6731 - accuracy: 0.5525 - val_loss: 1.0509 - val_accuracy: 0.5867
Epoch 2/300
1200/1200 [==============================] - 12s 10ms/step - loss: 0.9201 - accuracy: 0.6358 - val_loss: 0.8052 - val_accuracy: 0.6383
Epoch 3/300
1200/1200 [==============================] - 11s 9ms/step - loss: 0.8114 - accuracy: 0.6754 - val_loss: 0.7525 - val_accuracy: 0.6683
Epoch 4/300
1200/1200 [==============================] - 12s 10ms/step - loss: 0.6956 - accuracy: 0.7208 - val_loss: 0.6199 - val_accuracy: 0.7533
Epoch 5/300
1200/1200 [==============================] - 12s 10ms/step - loss: 0.6195 - accuracy: 0.7387 - val_loss: 0.6997 - val_accuracy: 0.7283
Epoch 6/300
1200/1200 [==============================] - 12s 10ms/step - loss: 0.5561 - accuracy: 0.7767 - val_loss: 0.6278 - val_accuracy: 0.7483
Epoch 7/300
1200/1200 [==============================] - 12s 10ms/step - loss: 0.4837 - accuracy: 0.8138 - val_loss: 0.

<h6>Some basic results recording, just for keeping information to hand</h6>

94.34445095062256 seconds --- 30 epochs , 0.9033 acc , 32x32 RGB image (images are still in greyscale, just RGB format)

343.3586163520813 seconds --- 30 epcohs , 0.9182 acc , 64x64 RGB image (images are still in greyscale, just RGB format)

1364.8851137161255 seconds --- 30 epochs , 0.8950 acc , 128x128 Greyscale image

313.81851744651794 seconds --- 30 epochs , 0.8750 acc , 64x64 Greyscale image

87.09897923469543 seconds --- 30 epochs , 0.8833 acc , 32x32 Greyscale image

31.2696533203125 seconds --- 30 epochs , 0.8200 acc , 16x16 Greyscale image

183.15469479560852 seconds --- 60 epochs , 0.9117 acc , 32x32 Greyscale image

<h5>Read in the validation data in the same way we read in the training and testing data</h5>

In [92]:
img_reshape_size = (32,32)

validation_data = []
validation_result = []

encoder = OneHotEncoder()
encoder.fit([[0], [1], [2], [3]]) 

# path to non tumor files
no_tumor_image_dir = "test_dataset/test/split_data/multi-class/no_tumor"
no_tumor_files = os.listdir(no_tumor_image_dir)

# path to meningioma files
meningioma_image_dir = "test_dataset/test/split_data/multi-class/meningioma_tumor"
meningioma_files = os.listdir(meningioma_image_dir)

# path to glioma files
glioma_image_dir = "test_dataset/test/split_data/multi-class/glioma_tumor"
glioma_files = os.listdir(glioma_image_dir)

# path to pitulitary files
pitulitary_image_dir = "test_dataset/test/split_data/multi-class/pitulitary_tumor"
pitulitary_files = os.listdir(pitulitary_image_dir)


for file in no_tumor_files:
    temp_file_path = no_tumor_image_dir + "/" +file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    validation_data.append(np.array(img))
    validation_result.append(encoder.transform([[0]]).toarray())

for file in meningioma_files:
    temp_file_path = meningioma_image_dir + "/" + file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    validation_data.append(np.array(img))
    validation_result.append(encoder.transform([[1]]).toarray())
        
for file in glioma_files:
    temp_file_path = glioma_image_dir + "/"  +file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    validation_data.append(np.array(img))
    validation_result.append(encoder.transform([[2]]).toarray())
        
        
for file in pitulitary_files:
    temp_file_path = pitulitary_image_dir + "/"  +file
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    validation_data.append(np.array(img))
    validation_result.append(encoder.transform([[3]]).toarray())
     

<h5>reshape validation data</h5>

In [93]:
validation_data = np.array(validation_data)
validation_result = np.array(validation_result)
validation_result = validation_result.reshape(200,4)

<h5>Get and print scores</h5>

In [94]:
scores = model.evaluate(validation_data, validation_result, verbose=10)

In [95]:
print("test loss: " + str(scores[0]))
print("test accuracy: " + str(scores[1]))

test loss: 0.25100213289260864
test accuracy: 0.9350000023841858
